## Timeline
**Start of Project:** 04.11.2019 
**Submission of Project Results:** 27.11.2019

**Phases:** 
1. Week (06.11 - 12.11): Preprocessing 
2. Week (13.11 - 19.11): Iterate & Evaluate the Models 
3. Week (20.11 - 26.11): Writing the Report

## Responsibilities
- Liste mit Begründungen für Auswahl der Feature (Feature | Begründung) - Jannik
- Preprocessing: 
  - Neighbourhood_cleansed (One-hot) (& PLZ Gibt es genug Beispiele prüfen?)- Daniel 
  - Property_type (One-hot) - Jannik 
  - Room_type (One-hot) - Jannik
  - Accomodates - Caro
  - Bathrooms (Maybe) - Caro
  - Bedrooms - Fritz
  - Amenities - Jakob
  - Beds - Fritz
  - Bed_type (One-hot) - Fritz

- Juli Datensatz fürs Preprocessing

# Airbnb Price Prediction

## Goal
The goal of our data mining project is to predict prices for new Airbnb listings in Munich. To achieve this, we will train a regression model on existing Airbnb data from www.insideairbnb.com.

## Import libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest, SelectFwe
from collections import Counter

## Load the data

In [28]:
munich = pd.read_csv('Data/listings_Munich_July.csv')

# drop all colums except the ones that we are using
munich = munich.loc[:, munich.columns.intersection(['neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price', 'amenities'])]

# convert column price to float values without leading $ sign
munich['price'] = munich['price'].replace( '[\$,)]','', regex=True ).astype(float)

C:\Users\jakob\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (43,61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Understand the data

In [74]:
#munich.head()

In [6]:
# understand feature neighbourhoods_cleansed
print(munich['neighbourhood_cleansed'].value_counts())

Ludwigsvorstadt-Isarvorstadt                              1077
Maxvorstadt                                               1000
Schwabing-West                                             735
Neuhausen-Nymphenburg                                      702
Au-Haidhausen                                              686
Schwabing-Freimann                                         589
Bogenhausen                                                471
Milbertshofen-Am Hart                                      428
Sendling                                                   427
Schwanthalerhöhe                                           409
Ramersdorf-Perlach                                         368
Sendling-Westpark                                          361
Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln     359
Laim                                                       353
Obergiesing                                                346
Untergiesing-Harlaching                                

In [7]:
# Fritz: understand feature bedrooms 
# total
print(munich['bedrooms'].value_counts().sort_index())
# normalized
print(munich['bedrooms'].value_counts(normalize=True).sort_index())

0.0     1023
1.0     7683
2.0     1157
3.0      288
4.0       57
5.0        4
6.0        2
7.0        1
8.0        6
10.0       1
20.0       1
25.0       1
Name: bedrooms, dtype: int64
0.0     0.100059
1.0     0.751467
2.0     0.113165
3.0     0.028169
4.0     0.005575
5.0     0.000391
6.0     0.000196
7.0     0.000098
8.0     0.000587
10.0    0.000098
20.0    0.000098
25.0    0.000098
Name: bedrooms, dtype: float64


One can see that most listings have one or two bedrooms. In 1023 cases there seems to be no bedroom. This value is set by Airbnb landlords if there is not seperated bedroom (e.g. a one room apartment or when accommodating in the living room). It needs to be evaluated how well several regression models can handle this encoding.

The listings with 25, 20 and 10 bedrooms are hostels. As there only exist 16 listings with more than four bedrooms, it might make sense to drop those as these are extreme outliers. We can later on specify that our model is only able to predict price for new listings with less than five bedrooms.

In [8]:
# Fritz: understand feature beds
# total
print(munich['beds'].value_counts().sort_index())
# normalized
print(munich['beds'].value_counts(normalize=True).sort_index())

0.0       48
1.0     6608
2.0     2455
3.0      575
4.0      279
5.0       98
6.0       83
7.0       28
8.0       31
9.0        1
10.0       3
12.0       1
14.0       1
15.0       1
16.0      10
18.0       1
20.0       1
30.0       1
50.0       1
Name: beds, dtype: int64
0.0     0.004694
1.0     0.646196
2.0     0.240074
3.0     0.056229
4.0     0.027283
5.0     0.009583
6.0     0.008117
7.0     0.002738
8.0     0.003031
9.0     0.000098
10.0    0.000293
12.0    0.000098
14.0    0.000098
15.0    0.000098
16.0    0.000978
18.0    0.000098
20.0    0.000098
30.0    0.000098
50.0    0.000098
Name: beds, dtype: float64


Here one can see a similar situation as with the number of bedrooms. Most listings have one or two beds. In 48 cases Airbnb landlords set the number of beds to zero. A manual check of some listings showed that beds are actually present and listings were not created correctly. Thus, listings with zero beds will not be considered in the training.

Only 21 listings offer 9 or more beds. These might be dropped as well like examples with five ore more bedrooms.

In [9]:
# Fritz: understand feature bed_type
# total
print(munich['bed_type'].value_counts())
# normalized
print(munich['bed_type'].value_counts(normalize=True))

Real Bed         9728
Pull-out Sofa     343
Couch              79
Futon              61
Airbed             27
Name: bed_type, dtype: int64
Real Bed         0.950186
Pull-out Sofa    0.033503
Couch            0.007716
Futon            0.005958
Airbed           0.002637
Name: bed_type, dtype: float64


One can see that real beds are offered in the great majority (95%) of all listings. Pull-out sofas occur in about 3% while listings with bed types couch, futon and airbed are not really present in the dataset.

Based on the assumption that there are no big differences in comfort when comparing pull-out sofa, couch, futon and airbed, those categorical values can be merged. A difference to real beds shall be considered. The result is a single boolean feature 'Real Bed' which only holds true and false values.

### Amenities

As they potentially provide significant value to the customers of Airbnb (e.g. kitchen, internet), amenities could easily affect the price of a given listing and are therefore included in the analysis.

First, as the amenities in the data are given as a list of strings per listing, we have to split the data in order to analyse which specific amenities might be useful to the analysis.

In [29]:
all_amenities = list()  #create list of all individual amenities
for amenities in munich['amenities']:
    for amenity in amenities.split("{")[1].split("}")[0].split(","):
        all_amenities.append(amenity.replace('"', ''))
#all_amenities_set = set(all_amenities)
Counter(all_amenities).most_common(50)

[('Heating', 9709),
 ('Wifi', 9581),
 ('Essentials', 9314),
 ('Kitchen', 9306),
 ('Washer', 7907),
 ('Smoke detector', 7127),
 ('Hair dryer', 6978),
 ('Hangers', 6748),
 ('Laptop friendly workspace', 6408),
 ('TV', 6300),
 ('Iron', 5990),
 ('Shampoo', 5617),
 ('Hot water', 4657),
 ('Elevator', 4490),
 ('Dryer', 3517),
 ('Lock on bedroom door', 3460),
 ('Refrigerator', 2983),
 ('Dishes and silverware', 2909),
 ('Internet', 2905),
 ('Host greets you', 2797),
 ('Bed linens', 2765),
 ('Stove', 2742),
 ('Cooking basics', 2718),
 ('Family/kid friendly', 2500),
 ('Cable TV', 2458),
 ('Oven', 2358),
 ('Buzzer/wireless intercom', 2217),
 ('First aid kit', 2200),
 ('Coffee maker', 2127),
 ('Dishwasher', 1988),
 ('Free parking on premises', 1933),
 ('Free street parking', 1864),
 ('Patio or balcony', 1706),
 ('Private entrance', 1649),
 ('Fire extinguisher', 1619),
 ('No stairs or steps to enter', 1529),
 ('Carbon monoxide detector', 1496),
 ('Extra pillows and blankets', 1467),
 ('Luggage dropof

We now can filter out those amenities which conceivably affect the price and which are occuring often enough in order to be regarded as significant.

We extract these features into a list and create new variables for each one to perfom One-Hot-Encoding.

In [30]:
relevant_amenities = ['Wifi', 'Internet', 'TV', 'Kitchen', 'Heating', 'Washer', 'Patio or balcony', 
                      'Breakfast', 'Elevator','24-hour check-in', 'Pool', 
                      'Private entrance','Dishwasher', 'Bed linens','Smoking allowed'] 

for element in relevant_amenities: #create dummy variables
    munich[element.lower()] = 0
    
for row in munich.itertuples(): #can take a moment
    for element in relevant_amenities:
        if element in row.amenities:
            munich.loc[row.Index, element.lower()] = 1
            
munich = munich.drop(columns = 'amenities')

In [31]:
for amenity in relevant_amenities: #analyse features with regard to districts
    print(pd.crosstab(munich['neighbourhood_cleansed'], munich[amenity.lower()]))
    print()

wifi                                                 0     1
neighbourhood_cleansed                                      
Allach-Untermenzing                                  1    60
Altstadt-Lehel                                      19   311
Au-Haidhausen                                       37   649
Aubing-Lochhausen-Langwied                           4    78
Berg am Laim                                        17   194
Bogenhausen                                         36   435
Feldmoching-Hasenbergl                               3    94
Hadern                                               8   116
Laim                                                27   326
Ludwigsvorstadt-Isarvorstadt                        71  1006
Maxvorstadt                                         59   941
Milbertshofen-Am Hart                               33   395
Moosach                                             10   183
Neuhausen-Nymphenburg                               46   656
Obergiesing             

The features 'internet' and 'wifi' appear to be redundant. Therefore, the latter can probably be discarded. Furthermore, there seems to be a relatively small number of 'pools' in the data which suggests that the variable should be filtered out as well.

Apart from that, it should be discussed whether the infrequent occurence of features such as 'bed linens' is due to hosts not giving the respective information correctly. 

## Preprocess the data

In [49]:
# one-hot encoding where necessary

## Compare different months

In [45]:
munich_09 = pd.read_csv('Data/listings_19_09.csv')
munich_09['price'] = munich_09['price'].replace( '[\$,)]','', regex=True ).astype(float)
munich_08 = pd.read_csv('Data/listings_19_08.csv')
munich_06 = pd.read_csv('Data/listings_19_06.csv')
munich_05 = pd.read_csv('Data/listings_19_05.csv')

In [47]:
print('May')
print('Median: ', munich_05['price'].median())
print('Mean: ', munich_05['price'].mean())
print()
print('June')
print('Median: ', munich_06['price'].median())
print('Mean: ', munich_06['price'].mean())
print()
print('July')
print('Median: ', munich['price'].median())
print('Mean: ', munich['price'].mean())
print()
print('August')
print('Median: ', munich_08['price'].median())
print('Mean: ', munich_08['price'].mean())
print()
print('September')
print('Median: ', munich_09['price'].median())
print('Mean: ', munich_09['price'].mean())


May
Median:  80.0
Mean:  107.91782208278514

June
Median:  80.0
Mean:  108.27851247887044

July
Median:  80.0
Mean:  109.05909357296348

August
Median:  80.0
Mean:  113.40779101741522

September
Median:  89.0
Mean:  119.22780610180465


One can see that the average prices are quite stabile from May to August but increase a lot in September. This can be traced back to the Oktoberfest in Munich that takes place in September. We need to find a good solution for dealing with these variations. 